This time series data is not consistent. The goal of the notebook is to use Linear Interpolation to generate consiste timestamps for every 5 minutes.

First I will generate more timestamps for evey 5 minute interval.

<a href = "https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra">How can I replace all the NaN values with Zero's in a column of a pandas dataframe</a>

In [3]:
def generate_filler_timestamps(df, year, step = 5):
    prev_date = df.iloc[0].name

    tdelta = datetime.timedelta(minutes = step)

    original_indicies = df.index.to_series()

    new_timestamps = []

    while (not prev_date == datetime.datetime(year, 12, 31, 23, 55)):
        if(prev_date in original_indicies):
            prev_date = prev_date + tdelta
            continue
        new_timestamps.append(prev_date)
        prev_date = prev_date + tdelta

    new_timestamps.append(datetime.datetime(year, 12, 31, 23,55,0))

    df2 = pd.DataFrame(index = new_timestamps, columns=["departure_terminal", "arrival_terminal",
                                                        "scheduled_departure", "actual_departure",
                                                        "Actual_Arrival", "TransitTime", "seconds_late"])
    df2["departure_terminal"].fillna("Bainbridge", inplace = True)
    df2["arrival_terminal"].fillna("Colman", inplace = True)
    df2.index.rename("timestamps", inplace = True)

    df3 = df.append(df2)
    df3 = df3.sort_values(by = "timestamps")
    
    return df3

In [4]:
import os
import pandas as pd
import datetime

for filename in os.listdir("../csv-files/Step 2 - Seconds Late/"):
    df = pd.read_csv("../csv-files/Step 2 - Seconds Late/%s" % (filename), parse_dates = [0], index_col = [0])
    df.index.name = "timestamps"
    year = filename.split("_")[2]
    dfLarge = generate_filler_timestamps(df, int(year))
    dfLarge["seconds_late"] = dfLarge["seconds_late"].interpolate()
    dfLarge.to_csv("../csv-files/Step 3 - Interpolated/Bainbridge_Departures_%s_Interpolated.csv" % year)